In [106]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from cleaning_data import *
from stats import *
from functions import *
from utils import split_data, downsample_class_0, upsample_class_1_to_percentage
from functions import *
import datetime
import seaborn as sns
from optimization.graphs import *
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
DATA_PATH = '/Users/williamjallot/Desktop/ML/dataset'
x_train, x_test, y_train, train_ids, test_ids, labels =  load_csv_data(DATA_PATH, sub_sample=False)
labels.pop(0) 

KeyboardInterrupt: 

In [56]:
#Split the data into training and testing
x_tr, x_te, y_tr, y_te = split_data(x_train, y_train, 0.8, seed= 2)

In [76]:

x_train_cleaned, y_tr_cleaned, features, median_and_most_probable_class, W, mean, x_before_outliers = clean_train_data(x_tr, y_tr,labels, 0.2, 1, 0.995, 1, 0.6)

In [81]:
# Assuming y_train is a numpy array
# Count the unique values and their frequencies
unique_classes, class_counts = np.unique(y_tr, return_counts=True)/y_tr.shape[0]

# Plot the class distribution
plt.figure(figsize=(6,4))
plt.bar(unique_classes, class_counts, color=['blue', 'red'])
plt.title('Class Distribution in y_train')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.xticks(unique_classes)
plt.savefig('images/data_imbalance.png')
plt.show()

/var/folders/fv/f56ltf0x04g0yp86bf3wx6w80000gn/T/ipykernel_2732/1825980835.py:13: UserWarning: Matplotlib is currently using pdf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [77]:
plt.figure(figsize=(12, 6))
sns.set_theme(style="whitegrid")

# Plot the box plot with Seaborn
sns.boxplot(data=x_before_outliers)

# Set plot labels and title
plt.title("Box Plot of All Features Before Outliers removal")
plt.xlabel("Features")
plt.ylabel("Value Range")
plt.gca().set_xticklabels([])
plt.show()
plt.savefig('images/before_outliers.png')

/var/folders/fv/f56ltf0x04g0yp86bf3wx6w80000gn/T/ipykernel_2732/604338762.py:12: UserWarning: Matplotlib is currently using pdf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [78]:
plt.figure(figsize=(12, 6))
sns.set(style="whitegrid")

# Plot the box plot with Seaborn
sns.boxplot(data=x_train_cleaned)

# Set plot labels and title
plt.ylim(-150, 150)
plt.title("Box Plot of All Features After Outliers removal")
plt.xlabel("Features")
plt.ylabel("Value Range")
plt.xticks([])
plt.show()
plt.savefig('images/after_outliers.png')

/var/folders/fv/f56ltf0x04g0yp86bf3wx6w80000gn/T/ipykernel_2732/2341763351.py:13: UserWarning: Matplotlib is currently using pdf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [125]:
# Define hyperparameters in a dictionary
hyperparameters = {
    "lambdas": [0.01],
    "up_sampling_percentages": [0.2],
    "degrees": [1],
    "variances_threshold": [0.995],
    "decision_threshold": [0.47],
    "acceptable_nan_percentages": [1],
    "max_iters": [300],
    "outliers_row_limit": [0.7],
    "gammas": [0.5],
    "nan_handlers": ['numeric', 'median', 'mean']
}

# Define k-fold and any other needed parameters
k_fold = 4

# Call the updated function
best_params = grid_search_k_fold_logistic(y_tr, x_tr, k_fold, hyperparameters, labels)

# Unpack the best parameters if you need to use them individually
best_gamma = best_params['gamma']
best_up_sampling_percentage = best_params['up_sampling_percentage']
best_degree = best_params['degree']
best_variance_threshold = best_params['variance_threshold']
best_lambda = best_params['lambda_']
best_max_iter = best_params['max_iter']
best_threshold = best_params['threshold']
best_nan_percentage = best_params['acceptable_nan_percentage']
best_outlier_limit = best_params['outlier_limit']
best_nan_handler = best_params['nan_handling']


Beginning grid search with k-fold cross-validation
Grid creation completed
Step 1/3
F1 score of 0.4058643528954553, gamma = 0.5, up_sampling_percentage = 0.2, degree = 1, variance_threshold = 0.995, lambda = 0.01, outlier limit = 0.7, max_iter = 300, threshold = 0.47, nan percentage = 1, nan handling = numeric


KeyboardInterrupt: 

In [26]:
print(f'The best parameters that yield a f1 score of {best_f1_score} are gamma = {best_gamma}, up_sampling_percentage = {best_up_sampling_percentage}, degree = {best_degree}, variance_treshold = {best_variance_threshold}, lambda = {best_lambda}, threshold = {best_threshold},max_iter = {best_max_iter}, best nan percentage = {best_nan_percentage}')

The best parameters that yield a f1 score of 0.414297040192062 are gamma = 0.5, up_sampling_percentage = 0.2, degree = 1, variance_treshold = 0.995, lambda = 0.01, threshold = 0.47,max_iter = 300, best nan percentage = 1


In [48]:
x_train_cleaned, y_tr_cleaned, features, median_and_most_probable_class, W, mean = clean_train_data(x_tr, y_tr,labels, best_up_sampling_percentage, best_degree, best_variance_threshold, best_nan_percentage, 0.6)

In [49]:
x_te_cleaned = clean_test_data(x_te, labels, features, median_and_most_probable_class, mean, W, best_degree)

In [50]:
# Parameters for the Gaussian distribution
mean = 0    # Mean of the distribution
std_dev = 1 # Standard deviation of the distribution

num_samples = x_train_cleaned.shape[0]
tx = np.c_[np.ones(num_samples), x_train_cleaned]
w_initial = np.random.normal(loc=mean, scale=std_dev, size=tx.shape[1])


w, loss = reg_logistic_regression(y_tr_cleaned, tx, best_lambda ,w_initial, best_max_iter, best_gamma)

In [51]:
num_samples = x_te_cleaned.shape[0]
tx_te = np.c_[np.ones(num_samples), x_te_cleaned]
y_predict = predict(tx_te, w, best_threshold)
f1_score_te = f1_score(y_te, y_predict)

In [52]:
print(f1_score_te)

0.4171835818925696


In [11]:
x_test_cleaned = clean_test_data(x_test, labels, features, median_and_most_probable_class, mean, W, best_degree)
num_samples = x_te.shape[0]
tx_te = np.c_[np.ones(num_samples), x_te_cleaned]

In [12]:
num_samples = x_test_cleaned.shape[0]
tx_test = np.c_[np.ones(num_samples), x_test_cleaned]
y_test_to_save = predict(tx_test, ws[-1], best_threshold)

In [13]:
# Stack the ids and predictions together column-wise
submit = np.column_stack((test_ids, y_test_to_save))

# Save to a CSV file using np.savetxt
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/sample_submission.csv", submit, delimiter=",", fmt='%d,%d', header='Id,Prediction', comments='')

The best parameters that yield a f1 score of 0.3947067098070202 are gamma = 0.5, up_sampling_percentage = 0.2, degree = 1, variance_treshold = 0.97, lambda = 0.01, max_iter = 300

The best parameters that yield a f1 score of 0.40757941432083533 are gamma = 0.45, up_sampling_percentage = 0.2, degree = 1, variance_treshold = 0.99, lambda = 0.01, threshold = 0.465,max_iter = 300, best nan percentage = 0.6

The best parameters that yield a f1 score of 0.40899345994682 are gamma = 0.45, up_sampling_percentage = 0.2, degree = 1, variance_treshold = 0.995, lambda = 0.01, threshold = 0.47,max_iter = 300, best nan percentage = 1